<a href="https://colab.research.google.com/github/falvojr/dio-lab-open-source/blob/main/PR_Labeler_DIO_Lab_Workflow_Trigger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import logging
from typing import List, Optional, Dict
import requests

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [ ]:
class GitHubConfig:
    TOKEN = os.getenv('GITHUB_TOKEN', 'token_placeholder')
    OWNER = os.getenv('GITHUB_OWNER', 'digitalinnovationone')
    REPO = os.getenv('GITHUB_REPO', 'dio-lab-open-source')
    HEADERS = {
        'Authorization': f'token {TOKEN}',
        'Accept': 'application/vnd.github.v3+json'
    }


In [ ]:
class ScriptConfig:
    DIO_LAB_WORKFLOW_TRIGGER_LABEL = 'run dio workflow'
    DELAY_SECONDS = 10


In [ ]:
class GitHubClient:
    @staticmethod
    def make_request(method: str, endpoint: str, json: Optional[Dict] = None) -> Optional[requests.Response]:
        url = f"https://api.github.com/repos/{GitHubConfig.OWNER}/{GitHubConfig.REPO}{endpoint}"
        try:
            response = requests.request(method, url, headers=GitHubConfig.HEADERS, json=json)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            logging.error(f"Erro ao fazer requisição {method} para {url}: {e}")
            return None

    @staticmethod
    def get_open_prs() -> List[Dict]:
        prs = []
        page = 1

        while True:
            response = GitHubClient.make_request("GET", f"/pulls?page={page}&per_page=100")
            if not response:
                break

            page_prs = response.json()
            prs.extend(page_prs)

            if len(page_prs) < 100:
                break
            page += 1
        logging.info(f"Recuperados {len(prs)} PRs abertos no total.")
        return prs

    @staticmethod
    def add_label(pr_number: int, label: str) -> bool:
        """Adiciona uma label a um PR."""
        data = {'labels': [label]}
        response = GitHubClient.make_request("POST", f"/issues/{pr_number}/labels", json=data)
        if response:
            logging.info(f"Adicionado label ao PR #{pr_number}. Resposta: {response.status_code}")
            return True
        return False


In [ ]:
def main():
    prs_abertos = GitHubClient.get_open_prs()
    prs_abertos.reverse()

    for pr in prs_abertos:
        pr_number = pr.get('number')
        labels = [label['name'] for label in pr.get('labels', [])]

        if ScriptConfig.DIO_LAB_WORKFLOW_TRIGGER_LABEL not in labels:
            if GitHubClient.add_label(pr_number, ScriptConfig.DIO_LAB_WORKFLOW_TRIGGER_LABEL):
                time.sleep(ScriptConfig.DELAY_SECONDS)
        else:
            logging.info(f"PR #{pr_number} já possui o label. Pulando.")

if __name__ == "__main__":
    main()
